# Building .NET Interactive [Optional]

The core functionality in Try .NET is the .NET Interactive repo, which contains the `CSharpProjectKernel` that powers Try .NET's compilation and language services. 

* If you're making changes in that layer, then you'll need to build the .NET Interactive packages locally. 

* If not, skip to step 2.1.

## 1.1: Clone and build .NET Interactive

First, make sure you have the [.NET Interactive repo](https://github.com/dotnet/interactive) forked and cloned. Set the path to the repo root in the following cell. 


In [ ]:
$dotnetInteractiveRepoPath = Read-Host -Prompt "Enter the path to the root of your dotnet/interactive clone"
$dotnetInteractiveRepoPath

Follow the instructions to build .NET Interactive in the [developer guide](https://github.com/dotnet/interactive/blob/main/DEVELOPER-GUIDE.md). 

You don't need to set up a local build of the Polyglot Notebooks extension to work on Try .NET.

The following script will build the project and copy the generated packages to C:\temp\packages\ which will be be used a NuGet package source when building Try .NET.

In [ ]:
Invoke-Expression "$dotnetInteractiveRepoPath/repack.ps1"

## 2.1: Clone and build Try .NET

In [3]:
$tryDotnetRepoPath = Get-Location

$tryDotNetProjectFilePath="$tryDotnetRepoPath\src\Microsoft.TryDotNet\Microsoft.TryDotNet.csproj"
$mockMsLearnServerPath="$tryDotnetRepoPath\src\microsoft-learn-mock"

In [ ]:
if ($IsWindows) {
    Invoke-Expression "$tryDotnetRepoPath\build-js.cmd"
}else {
    Invoke-Expression "$tryDotnetRepoPath/build-js.sh"
}

dotnet build

## 2.2: [Optional] Update Microsoft.DotNet.Interactive.CSharpProject version used by Try .NET

This step only needs to be done if you're changing code from the .NET Interactive repo.

This step modifies NuGet.config to reference the local package source where .NET Interactive packages build in Step 1 should be found.

In [ ]:
$nugetConfigPath = Join-Path $tryDotnetRepoPath 'NuGet.config'
[xml]$nugetConfig = Get-Content $nugetConfigPath

$newPackageSource = $nugetConfig.CreateElement("add")
$newPackageSource.SetAttribute("key", "LocalPackages")
$newPackageSource.SetAttribute("value", "C:\temp\packages")

$packageSourcesNode = $nugetConfig.SelectSingleNode('configuration/packageSources')
$packageSourcesNode.AppendChild($newPackageSource) > $null

$packageMappingNode = $nugetConfig.CreateElement("packageSource")
$packageMappingNode.SetAttribute("key", "temp")
$packageNode = $nugetConfig.CreateElement("package")
$packageNode.SetAttribute("pattern", "*")
$packageMappingNode.AppendChild($packageNode) > $null

$packageSourcesNode = $nugetConfig.SelectSingleNode('configuration/packageSourceMapping')
$packageSourcesNode.AppendChild($packageMappingNode) > $null

$nugetConfig.Save($nugetConfigPath)

Next, update the referenced version of the package.

In [ ]:
# Define the path to the Directory.Packages.props file
$packagesPropsPath = "$tryDotnetRepoPath\Directory.Packages.props"

# Load the XML content of the Directory.Packages.props file
[xml]$packagesProps = Get-Content $packagesPropsPath

# Define the namespace to handle the default namespace in the document
$namespaceManager = New-Object System.Xml.XmlNamespaceManager($packagesProps.NameTable)
$namespaceManager.AddNamespace("msb", "http://schemas.microsoft.com/developer/msbuild/2003")

# Ensure that the ItemGroup element exists
$itemGroup = $packagesProps.SelectSingleNode("msb:Project/msb:ItemGroup", $namespaceManager)
if ($itemGroup -eq $null) {
    $itemGroup = $packagesProps.CreateElement("ItemGroup", $namespaceManager.LookupNamespace("msb"))
    $packagesProps.DocumentElement.AppendChild($itemGroup)
}

# Check if the PackageVersion element for the package already exists
$packageRef = $itemGroup.SelectSingleNode("msb:PackageVersion[@Include='Microsoft.DotNet.Interactive.CSharpProject']", $namespaceManager)

if ($packageRef -eq $null) {
    # If the package reference doesn't exist, create it
    $newPackageRef = $packagesProps.CreateElement("PackageVersion", $namespaceManager.LookupNamespace("msb"))
    $newPackageRef.SetAttribute("Include", "Microsoft.DotNet.Interactive.CSharpProject")
    $newPackageRef.SetAttribute("Version", "2.0.0")
    $itemGroup.AppendChild($newPackageRef)
} else {
    # If it exists, just update the version
    $packageRef.SetAttribute("Version", "2.0.0")
}

# Save the changes back to the Directory.Packages.props file
$packagesProps.Save($packagesPropsPath)


## 2.3: Start the Try .NET service

This launches the Try .NET web service. It will launch in a separate terminal window. Closing that window will stop the server.

In [4]:
Start-Process pwsh -ArgumentList "-c dotnet run --no-build --project ${tryDotNetProjectFilePath} --launch-profile TryDotNet-Development"

## 2.5: [One time only] Build MS Learn mock server

In [ ]:
Set-Location -Path $mockMsLearnServerPath
npm ci
npm run buildProd

## 2.4: Verify Try .NET by running Peaky tests

In [5]:
#r "nuget:Peaky.Client"

using Peaky.Client;
using System.Net.Http;

var peakyClient = new PeakyClient(new Uri("https://localhost:7061/tests"));

var tests = (await peakyClient.GetTestsAsync(environment: "Development")) 
    .Where(t => t.Tags.Any(tag => tag == "kernel"));
    
foreach (var test in tests)
{
    var result = await test.GetResultAsync();
    result.Display();
}

Installed Packages Peaky.Client, 4.0.79

✅ Passed: Can_get_signature_help 
 🧪 https://localhost:7061/tests/Development/trydotnet/Can_get_signature_help 
 
 Application: trydotnet
 
 Environment: Development
 
 Tags: kernel
 
 {
 "Outcome": "passed",
 "ReturnValue": null,
 "Passed": true,
 "Log": "",
 "Duration": "00:00:01.6782308",
 "Exception": null,
 "Test": {
 "Application": "trydotnet",
 "Name": "Can_get_signature_help",
 "Environment": "Development",
 "Url": "https://localhost:7061/tests/Development/trydotnet/Can_get_signature_help",
 "Tags": [
 "kernel"
 ]
 }
}

✅ Passed: Console_prebuild_is_ready 
 🧪 https://localhost:7061/tests/Development/trydotnet/Console_prebuild_is_ready 
 
 Application: trydotnet
 
 Environment: Development
 
 Tags: kernel
 
 {
 "Outcome": "passed",
 "ReturnValue": {
 "EnableBuild": false,
 "Directory": null,
 "Name": "console",
 "EntryPointAssemblyPath": null,
 "TargetFramework": "net8.0"
 },
 "Passed": true,
 "Log": "",
 "Duration": "00:00:00.0447764",
 "Exception": null,
 "Test": {
 "Application": "trydotnet",
 "Name": "Console_prebuild_is_ready",
 "Environment": "Development",
 "Url": "https://localhost:7061/tests/Development/trydotnet/Console_prebuild_is_ready",
 "Tags": [
 "kernel"
 ]
 }
}

## 2.5: Launch the MS Learn mock server

This site hosts the Try .NET IFRAME and provides controls so you can see if things are working.

In [ ]:
Set-Location -Path $mockMsLearnServerPath

Start-Process pwsh -ArgumentList "-c npx http-server -p 7063"

## 2.6: Open the MS Learn mock site and interact with Try .NET

In [ ]:
Start-Process "http://127.0.0.1:7063/site/DocsHost.html?trydotnetUrl=https://localhost:7061/api/trydotnet.min.js&trydotnetOrigin=https://localhost:7061/"

## Try .NET / Microsoft Learn flow diagram

The diagram available through the link below provides a comprehensive of the interaction between Try .NET service (trydotnet.microsoft.com) and a host page (learn.microsoft.com).

https://github.com/dotnet/interactive/tree/main/src/Microsoft.DotNet.Interactive.CSharpProject#try-net--microsoft-learn-flow

The core of Try .NET is powered by the `CSharpProjectKernel`, which serves as the backend for .NET Interactive.